In [29]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

In [30]:
# get list of urls for players stats
players_url = 'https://www.basketball-reference.com/players/'
r = requests.get(players_url)
bs = BeautifulSoup(r.text)
r.ok

False

In [24]:
# get all last name directory links
    # links found in 'ul' tag with 'page_index' class
    # directory links will have a text with length 1 (e.g. "A")
    # base url must be added to directory path
letter_links = bs.find('ul', {'class': 'page_index'}).find_all('a')
# letter_links = ['https://www.basketball-reference.com' + link.get('href') for link in letter_links if len(link.text) == 1]
letter_links = ['https://www.basketball-reference.com' + link.get('href') for link in letter_links if link.text == 'F']
print(len(letter_links))

1


In [25]:
# get all player links from last name directory links

# initialize empty list of player links
player_links = []

# player links found in 'th' tag w/specific scope and class
# active players with 'strong' tag
# add base url
for letter_link in letter_links:
    time.sleep(2.5)
    r = requests.get(letter_link)
    bs = BeautifulSoup(r.text)
    player_tags = bs.find('tbody').find_all('th', {'scope': 'row', 'class': 'left'})
    player_links += ['https://www.basketball-reference.com' + player_tag.find('a').get('href') for player_tag in player_tags if player_tag.find('strong')]

print(len(player_links))

23


In [26]:
# build dataframe of all players' stats by season

# initialize empty list to which dfs will be appended
appended_data = []

for player_link in tqdm(player_links):

    time.sleep(4)

    # get tables from player page
    dfs = pd.read_html(player_link)
    # save df as second table in html table code
    df = dfs[0]

    # add player name as col
    r = requests.get(player_link)
    bs = BeautifulSoup(r.text)
    name = bs.find("div", {"id": "info", "class": "players"}).find("h1").text
    name = name.split('\n')[1]

    df['Name'] = name

    # append individual df to list
    appended_data.append(df)

# combine all dfs together
nba = pd.concat(appended_data)

 65%|██████▌   | 15/23 [01:00<00:32,  4.01s/it]


HTTPError: HTTP Error 429: Too Many Requests

In [6]:
# move Name to be beginning col
first = nba.pop('Name')
nba.insert(0, 'Name', first)

# drop all rows where Age is NaN (these are not specific years, could be career or specific team average)
nba = nba.dropna(subset=['Age'])

# reset index
nba = nba.reset_index(drop=True)

In [7]:
nba.head()

,Name,Exp,Season,Age,Tm,Lg,Pos,G,GS,MP,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Tari Eason,1,2022-23,21.0,HOU,NBA,PF,15,0,18.1,...,0.81,2.1,2.9,4.9,1.1,1.5,0.5,0.9,2.3,8.3
1,Jaime Echenique,1,2021-22,24.0,WAS,NBA,C,1,0,3.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Anthony Edwards,1,2020-21,19.0,MIN,NBA,SG,72,55,32.1,...,0.776,0.8,3.8,4.7,2.9,1.1,0.5,2.2,1.8,19.3
3,Anthony Edwards,2,2021-22,20.0,MIN,NBA,SG,72,72,34.3,...,0.786,0.9,3.9,4.8,3.8,1.5,0.6,2.6,2.3,21.3
4,Anthony Edwards,3,2022-23,21.0,MIN,NBA,SG,15,15,36.2,...,0.656,0.5,5.3,5.8,3.9,1.1,0.3,2.9,3.0,21.9


In [8]:
nba.shape

(103, 32)

In [9]:
# check to see # unique players, should be 859
nba.Name.nunique()

17

In [20]:
# read csv's, concatatenate, write out
all = pd.read_csv('nba.csv')

all = all.drop(columns='Exp')

nba = pd.concat([all, nba])

nba.to_csv('nba.csv', index=False)

In [ ]:
# 11-18
# A-D, next up: F